
# *Exploration of Taxi Trip Fares in the San Francisco Bay Area*

In [8]:
from datascience import *
import numpy as np
import matplotlib.pyplot as plt
import warnings
warnings.filterwarnings('ignore')
%matplotlib inline

In [43]:
taxi_data = Table.read_table("SF_taxi_data.csv")

### Relationship Between Fares and Distance

According to the SFMTA fare calculation table, <br>
$Fare(x) \geq 3.5 + 0.55 × (5x − 1)$ <br>
We will analyze how this model compares to the actual relationship between fares and distance using real data.

In [55]:
predicted_fare = 3.5 + 0.55 * (5 * taxi_data.column("dist (miles)") - 1)
actual_fare = taxi_data.column("fare ($)")
least_squared_error = sum((actual_fare - predicted_fare) ** 2) / len(actual_fare)
least_squared_error

51.760001810791501

Since the least squared error calculated from this is 

### Analysis of Trips Related to SFO

In order to split up the total taxi trips to trips that include SFO and trips that do not include SFO, let us divide the taxi_data table into two tables that fit the criteria.

In [53]:
sfo_taz = taxi_data.group("deptaz").sort("count", descending=True).column("deptaz").item(0)
sfo_labeled_table = taxi_data.with_column("sfo", np.logical_or(taxi_data.column("deptaz") == np.full(taxi_data.num_rows, sfo_taz), taxi_data.column("arrtaz") == np.full(taxi_data.num_rows, sfo_taz)))
sfo_trips = sfo_labeled_table.where("sfo", True).drop("sfo")
no_sfo_trips = sfo_labeled_table.where("sfo", False).drop("sfo")

Let us analyze the differences between trips that include SFO and trips that do not by looking at the distances and fares.

In [54]:
print("Mean distance for sfo trips : ", np.mean(sfo_trips.column("dist (miles)")))
print("Mean distance for non-sfo trips : ", np.mean(no_sfo_trips.column("dist (miles)")))
print("Mean fare for sfo trips : ", np.mean(sfo_trips.column("fare ($)")))
print("Mean fare for non-sfo trips : ", np.mean(no_sfo_trips.column("fare ($)")))

Mean distance for sfo trips :  13.6399973334
Mean distance for non-sfo trips :  2.14582874527
Mean fare for sfo trips :  49.0883790447
Mean fare for non-sfo trips :  12.3978968537


From these means, we can see that the distances for trips including SFO are on average much bigger than other trips, hence the average fare is also higher. This logically makes sense because SFO is quite far from most hotspots in the Bay Area.